In [1]:
from tkinter.tix import Tree
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import numpy as np
from io import BytesIO
engine = create_engine("sqlite:///Celeb_DB_v1.db", echo=True)
import io



from sqlalchemy import Column, ForeignKey, Integer, String, BLOB, LargeBinary
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base


import cv2 
import face_recognition
import os

In [4]:
Session = sessionmaker(bind=engine)
session = Session()


Base = declarative_base()

## for creating images table
class Images(Base):
    __tablename__ = "images"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
## for creating faces table
class Faces(Base):
    __tablename__ = "faces"
    id = Column(Integer, primary_key=True)
    image_id = Column(Integer)
    faces = Column(LargeBinary)
    person_id = Column(Integer)
    locations = Column(LargeBinary)

# crearing database
Base.metadata.create_all(engine)


# for entering a value in images table
def create_image(name):
    user = Images(name=name)
    session.add(user)
    session.commit()    

    #print("Image id is: ", user.id)
    return user.id

# for entering a value in faces table
def create_face(image_id, faces, person_id,locations):
    faces = Faces(image_id=image_id, faces=faces, person_id=person_id,locations=locations)
    session.add(faces)
    session.commit()    

    #print("Face id is: ", faces.id)
    return faces.id

# for coverting blob back to np array
def convert_array(text):
    return np.load(BytesIO(text), allow_pickle=True)


# find number of unique faces 
def Unique():
    U=[]
    for face in session.query(Faces).all():
        
        if face.person_id not in U:
            
            U.append(face.person_id)
        
    return len(U)


# generate list of images name with same person in it
def same_faces(Person_Ids):
    
    Same={}
    
    for id in Person_Ids:
        List=[]
        for face in session.query(Faces).filter(Faces.person_id==id):
            
            for img in session.query(Images).filter(Images.id==face.image_id):

                List.append(img.name)

                break



        #print(len(List))
        #print(List)

        Same.update({id:List})

    return Same 


# most frequent element in list
def most_frequent(List):
    return max(set(List), key = List.count)


# add to database using media pipe detector and dlib feature extractor for recognition
def ADD_FACE_dlib(file,path,show_list):
        
    Path=path+"/"+file 

    known_image = cv2.imread(Path)

    if known_image is None:

        print(file+ " Image Not Found")
        
    else:

        known_image = cv2.cvtColor(known_image, cv2.COLOR_BGR2RGB)

        size=np.shape(known_image)

        image_id=create_image(file)

        if (size[0]>1500 or size[1]>1500):
            known_image = cv2.resize(known_image, (0, 0), fx=0.25, fy=0.25)
        elif (size[0]>1000 or size[1]>1000):
            known_image = cv2.resize(known_image, (0, 0), fx=0.5, fy=0.5)
        elif (size[0]>750 and size[1]>750):
            known_image = cv2.resize(known_image, (0, 0), fx=0.75, fy=0.75)
            
        #locations = face_recognition.face_locations(known_image,0,model='cnn')

        locations = [face_recognition.api._trim_css_to_bounds(face_recognition.api._rect_to_css(face.rect), known_image.shape) for face in face_recognition.api._raw_face_locations(known_image, 1, "cnn") if face.confidence >0.9]
        
        if len(locations)==0:

            print("Face Not Found in "+file)

            #create_face(image_id, None, 0)

        else:
            
            #locations=face_locations_mp(results,known_image)

            encodings = face_recognition.face_encodings(known_image,locations,num_jitters=100)
            
            #start_time=time.time()

            for E,L in zip(encodings,locations):

                np_bytes = io.BytesIO()
                
                np_bytes_1 = io.BytesIO()

                np.save(np_bytes, E, allow_pickle=True)
                
                np.save(np_bytes_1, L, allow_pickle=True)

                c=0

                if len(session.query(Faces).all())==0:

                    print("Database is Empty... Creating First Entry")

                    create_face(image_id, np_bytes.getvalue(), 1,np_bytes_1.getvalue())

                    continue

                Matched={}
                
                End=False
                
                for face in session.query(Faces).all():

                    results= face_recognition.compare_faces([E],convert_array(face.faces) ,tolerance=0.5)

                    if results[0]==True:
                        
                        if face.person_id in Matched.keys():
                            
                            Matched[face.person_id]+=1
                            
                        else:
                        
                            Matched.update({face.person_id:1})
                    
                    for key in Matched:
                        
                        if Matched[key]>2:
                        
                            End=True
                            
                            break
                    
                    if End==True:
                        
                        break
                        
                #print(Matched)
                                       
                if len(Matched)==0:

                    P_id=Unique()+1

                    create_face(image_id, np_bytes.getvalue(), P_id,np_bytes_1.getvalue())

                    if show_list:

                        print(same_faces([P_id]))

                else:
                    
                    n=0
                    
                    for key in Matched:
                        
                        if Matched[key]>2:

                            P_id=key

                            create_face(image_id, np_bytes.getvalue(), P_id,np_bytes_1.getvalue())
                            
                            if show_list:

                                print(same_faces([P_id]))
                                
                            break
                                
                        else:
                                
                            n+=1
                                
                            
                    Max=-1
                                
                    if n==len(Matched):
                        
                        for key in Matched:
                            
                            if Matched[key]>Max:
                                
                                Max=key
                                
                                
                        if len(same_faces([Max])[Max])<4 and Matched[Max]==1:
                        
                        
                            P_id=Max

                            create_face(image_id, np_bytes.getvalue(), P_id,np_bytes_1.getvalue())
                        
                        elif len(same_faces([Max])[Max])<6 and Matched[Max]>1:
                            
                            P_id=Max
                            
                            create_face(image_id, np_bytes.getvalue(), P_id,np_bytes_1.getvalue())
                        
                        else:
                            
                            P_id=Unique()+1
                            
                            create_face(image_id, np_bytes.getvalue(), P_id,np_bytes_1.getvalue())
                            
                            
                        if show_list:

                            print(same_faces([P_id]))
                        
                            
            #print("--- %s seconds ---" % (time.time() - start_time))




2022-06-03 11:42:52,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:42:52,979 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("images")
2022-06-03 11:42:52,980 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-03 11:42:52,981 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("faces")
2022-06-03 11:42:52,982 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-03 11:42:52,983 INFO sqlalchemy.engine.Engine COMMIT


# List of Files in a Folder

In [5]:
Files=[]
import os
for file in os.listdir("Data"):
    #print(os.path.join("/mydir", file))
    Files.append(file)
    
len(Files)

100

# Add full List of Images in Database

In [6]:
import time 

start_time=time.time()

for file in Files:
    ADD_FACE_dlib(file,"Data",False)
    
print("--- %s seconds ---" % (time.time() - start_time))

2022-06-03 11:42:59,994 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:42:59,996 INFO sqlalchemy.engine.Engine INSERT INTO images (name) VALUES (?)
2022-06-03 11:42:59,997 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('Aaron0.jpg',)
2022-06-03 11:42:59,999 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:43:00,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:43:00,133 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:43:00,134 INFO sqlalchemy.engine.Engine [generated in 0.00106s] (1,)
2022-06-03 11:43:02,244 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:43:02,245 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ()
Database is Empty... Creating First Entry
2022-06-03 11:43:02,247 INFO sql

2022-06-03 11:43:03,779 INFO sqlalchemy.engine.Engine [cached since 3.646s ago] (4,)
2022-06-03 11:43:04,043 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:43:04,043 INFO sqlalchemy.engine.Engine [cached since 1.799s ago] ()
2022-06-03 11:43:04,044 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:43:04,045 INFO sqlalchemy.engine.Engine [cached since 1.801s ago] ()
2022-06-03 11:43:04,047 INFO sqlalchemy.engine.Engine INSERT INTO faces (image_id, faces, person_id, locations) VALUES (?, ?, ?, ?)
2022-06-03 11:43:04,048 INFO sqlalchemy.engine.Engine [cached since 1.801s ago] (4, <memory at 0x0000024CF98FE4C8>, 1, <memory at 0x0000024CF98FE108>)
2022-06-03

2022-06-03 11:43:06,737 INFO sqlalchemy.engine.Engine [cached since 4.494s ago] ()
2022-06-03 11:43:06,739 INFO sqlalchemy.engine.Engine INSERT INTO faces (image_id, faces, person_id, locations) VALUES (?, ?, ?, ?)
2022-06-03 11:43:06,740 INFO sqlalchemy.engine.Engine [cached since 4.494s ago] (8, <memory at 0x0000024CF98FE588>, 1, <memory at 0x0000024CF98FE4C8>)
2022-06-03 11:43:06,741 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:43:06,866 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:43:06,867 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces 
WHERE faces.id = ?
2022-06-03 11:43:06,867 INFO sqlalchemy.engine.Engine [cached since 4.464s ago] (8,)
2022-06-03 11:43:06,869 INFO sqlalchemy.engine.Engine INSERT INTO images (name) VALUES (?)
2022-06-03 11:43:06,870 INFO sqlalchemy.engine.Engine [cached since 6.873s ago

2022-06-03 11:43:08,104 INFO sqlalchemy.engine.Engine [cached since 5.217s ago] (7,)
2022-06-03 11:43:08,105 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:43:08,106 INFO sqlalchemy.engine.Engine [cached since 5.219s ago] (8,)
2022-06-03 11:43:08,107 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:43:08,107 INFO sqlalchemy.engine.Engine [cached since 5.22s ago] (9,)
2022-06-03 11:43:08,108 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:43:08,108 INFO sqlalchemy.engine.Engine [cached since 5.864s ago] ()
2022-06-03 11:43:08,109 INFO sqlalchemy.engine.Engine INSERT INTO faces (image_id, faces, person_id, locations) VALUES (?, ?, ?, ?)
2022-06-03 

2022-06-03 11:43:10,067 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces 
WHERE faces.id = ?
2022-06-03 11:43:10,068 INFO sqlalchemy.engine.Engine [cached since 7.665s ago] (13,)
2022-06-03 11:43:10,074 INFO sqlalchemy.engine.Engine INSERT INTO images (name) VALUES (?)
2022-06-03 11:43:10,075 INFO sqlalchemy.engine.Engine [cached since 10.08s ago] ('AaronTylor16.jpg',)
2022-06-03 11:43:10,076 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:43:10,214 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:43:10,217 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:43:10,219 INFO sqlalchemy.engine.Engine [cached since 10.09s ago] (14,)
2022-06-03 11:43:10,543 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_im

...

2022-06-03 11:43:55,215 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:43:55,216 INFO sqlalchemy.engine.Engine [cached since 52.33s ago] (81,)
2022-06-03 11:43:55,216 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:43:55,217 INFO sqlalchemy.engine.Engine [cached since 52.33s ago] (82,)
2022-06-03 11:43:55,218 INFO sqlalchemy.engine.Engine INSERT INTO faces (image_id, faces, person_id, locations) VALUES (?, ?, ?, ?)
2022-06-03 11:43:55,218 INFO sqlalchemy.engine.Engine [cached since 52.97s ago] (83, <memory at 0x0000024CF98FE708>, 13, <memory at 0x0000024CF98FE948>)
2022-06-03 11:43:55,220 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:43:55,357 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:43:55,358 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, fa

2022-06-03 11:43:57,804 INFO sqlalchemy.engine.Engine [cached since 55.56s ago] (87, <memory at 0x0000024CF98FE708>, 13, <memory at 0x0000024CF98FE948>)
2022-06-03 11:43:57,806 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:43:58,007 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:43:58,008 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces 
WHERE faces.id = ?
2022-06-03 11:43:58,009 INFO sqlalchemy.engine.Engine [cached since 55.61s ago] (88,)
2022-06-03 11:43:58,014 INFO sqlalchemy.engine.Engine INSERT INTO images (name) VALUES (?)
2022-06-03 11:43:58,015 INFO sqlalchemy.engine.Engine [cached since 58.02s ago] ('Alex37.jpg',)
2022-06-03 11:43:58,016 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:43:58,165 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:43:58,168 INFO sqlalchemy.engine.Engine SELECT i

2022-06-03 11:44:00,845 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:44:00,846 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces 
WHERE faces.id = ?
2022-06-03 11:44:00,846 INFO sqlalchemy.engine.Engine [cached since 58.44s ago] (92,)
2022-06-03 11:44:00,849 INFO sqlalchemy.engine.Engine INSERT INTO images (name) VALUES (?)
2022-06-03 11:44:00,849 INFO sqlalchemy.engine.Engine [cached since 60.85s ago] ('Alexandra33.jpg',)
2022-06-03 11:44:00,850 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:44:01,053 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:44:01,056 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:01,058 INFO sqlalchemy.engine.Engine [cached since 60.93s ago] (92,)
2022-06-03 11:44:01,333 INFO sqlalchemy

2022-06-03 11:44:03,128 INFO sqlalchemy.engine.Engine [cached since 60.73s ago] (95,)
2022-06-03 11:44:03,191 INFO sqlalchemy.engine.Engine INSERT INTO images (name) VALUES (?)
2022-06-03 11:44:03,192 INFO sqlalchemy.engine.Engine [cached since 63.2s ago] ('Alexandra42.jpg',)
2022-06-03 11:44:03,194 INFO sqlalchemy.engine.Engine COMMIT
2022-06-03 11:44:03,380 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-03 11:44:03,380 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:03,381 INFO sqlalchemy.engine.Engine [cached since 63.25s ago] (95,)
2022-06-03 11:44:03,684 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:44:03,684 INFO sqlalchemy.engine.Engine [cached since 61.44s ago] ()
2022-06-03 11:44:03,687 INFO sqlalchemy.engine.E

2022-06-03 11:44:06,371 INFO sqlalchemy.engine.Engine [cached since 66.24s ago] (99,)
2022-06-03 11:44:06,653 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:44:06,653 INFO sqlalchemy.engine.Engine [cached since 64.41s ago] ()
2022-06-03 11:44:06,655 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:44:06,656 INFO sqlalchemy.engine.Engine [cached since 64.41s ago] ()
2022-06-03 11:44:06,676 INFO sqlalchemy.engine.Engine INSERT INTO faces (image_id, faces, person_id, locations) VALUES (?, ?, ?, ?)
2022-06-03 11:44:06,676 INFO sqlalchemy.engine.Engine [cached since 64.43s ago] (99, <memory at 0x0000024CF98FE708>, 14, <memory at 0x0000024CF98FE948>)
2022-06

In [7]:
Unique()

2022-06-03 11:44:53,235 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:44:53,235 INFO sqlalchemy.engine.Engine [cached since 111s ago] ()


14

# Get list of images names with same persons

In [8]:
faces=same_faces(range(1,Unique()+1))
faces

2022-06-03 11:44:56,939 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces
2022-06-03 11:44:56,940 INFO sqlalchemy.engine.Engine [cached since 114.7s ago] ()
2022-06-03 11:44:56,943 INFO sqlalchemy.engine.Engine SELECT faces.id AS faces_id, faces.image_id AS faces_image_id, faces.faces AS faces_faces, faces.person_id AS faces_person_id, faces.locations AS faces_locations 
FROM faces 
WHERE faces.person_id = ?
2022-06-03 11:44:56,943 INFO sqlalchemy.engine.Engine [cached since 114.1s ago] (1,)
2022-06-03 11:44:56,945 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:56,946 INFO sqlalchemy.engine.Engine [cached since 114.1s ago] (1,)
2022-06-03 11:44:56,947 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM 

2022-06-03 11:44:56,996 INFO sqlalchemy.engine.Engine [cached since 114.1s ago] (26,)
2022-06-03 11:44:56,997 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:56,998 INFO sqlalchemy.engine.Engine [cached since 114.1s ago] (27,)
2022-06-03 11:44:56,999 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:56,999 INFO sqlalchemy.engine.Engine [cached since 114.1s ago] (28,)
2022-06-03 11:44:57,000 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,000 INFO sqlalchemy.engine.Engine [cached since 114.1s ago] (29,)
2022-06-03 11:44:57,001 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,002 INFO sqlalchemy.engine.Engine [cached since 114.

2022-06-03 11:44:57,044 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (55,)
2022-06-03 11:44:57,045 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,046 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (56,)
2022-06-03 11:44:57,046 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,047 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (57,)
2022-06-03 11:44:57,048 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,048 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (58,)
2022-06-03 11:44:57,049 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,050 INFO sqlalchemy.engine.Engine [cached since 114.

2022-06-03 11:44:57,091 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (86,)
2022-06-03 11:44:57,092 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,092 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (87,)
2022-06-03 11:44:57,093 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,093 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (88,)
2022-06-03 11:44:57,094 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,095 INFO sqlalchemy.engine.Engine [cached since 114.2s ago] (89,)
2022-06-03 11:44:57,096 INFO sqlalchemy.engine.Engine SELECT images.id AS images_id, images.name AS images_name 
FROM images 
WHERE images.id = ?
2022-06-03 11:44:57,096 INFO sqlalchemy.engine.Engine [cached since 114.

{1: ['Aaron0.jpg',
  'Aaron10.jpg',
  'Aaron12.jpg',
  'Aaron13.jpg',
  'Aaron3.jpg',
  'Aaron4.jpg',
  'Aaron6.jpg',
  'Aaron7.jpg',
  'Aaron8.jpg'],
 2: ['Aaron9.jpg'],
 3: ['AaronTylor12.jpg',
  'AaronTylor15.jpg',
  'AaronTylor16.jpg',
  'AaronTylor19.jpg',
  'AaronTylor22.jpg',
  'AaronTylor24.jpg',
  'AaronTylor25.jpg',
  'AaronTylor26.jpg'],
 4: ['AaronTylor13.jpg'],
 5: ['AaronTylor21.jpg'],
 6: ['Abigail1.jpg',
  'Abigail10.jpg',
  'Abigail11.jpg',
  'Abigail13.jpg',
  'Abigail3.jpg',
  'Abigail4.jpg',
  'Abigail5.jpg',
  'Abigail6.jpg',
  'Abigail7.jpg',
  'Abigail9.jpg'],
 7: ['Adele10.jpg',
  'Adele12.jpg',
  'Adele17.jpg',
  'Adele22.jpg',
  'Adele25.jpg',
  'Adele28.jpg',
  'Adele32.jpg',
  'Adele36.jpg',
  'Adele37.jpg',
  'Adele9.jpg'],
 8: ['Adrein10.jpg',
  'Adrein11.jpg',
  'Adrein12.jpg',
  'Adrein15.jpg',
  'Adrein16.jpg',
  'Adrein18.jpg',
  'Adrein22.jpg',
  'Adrein23.jpg',
  'Adrein25.jpg',
  'Adrein26.jpg'],
 9: ['Adrein25.jpg'],
 10: ['Adriana11.jpg',
  'Adria

In [9]:
Total=0
for key in faces:
    Total+=len(faces[key])
    
Total

101

# Visualize Based on Person_id

In [11]:
person_id=1
path='Data/'

for img in faces[person_id]:   
    Path= path+img  
    image = cv2.imread(Path)
    cv2.imshow('1',image)
    cv2.waitKey(500)
    cv2.destroyAllWindows()

#  Filter based on total images that contain a same person

In [12]:
counter=0

for key in faces:
    if len(faces[key])<2:
        #print(key)
        counter+=1

print(counter)

4


# Copying Images from main directory to folders with person id as a name

In [13]:
source="Data/"
destination="Celeb_v1/"

import shutil

for key in faces:
    
    path= destination + str(key)
    
    if len(faces[key])>2:
    
        os.mkdir(path)

        for file in faces[key]:

            source1 = source +file

            destination1 = path+"/"+file

            shutil.copy(source1, destination1)



# Draw Bounding Boxes and Visualize

In [ ]:
for i in range(1,Unique()+1):
    for face in session.query(Faces).filter(Faces.person_id == i):
        location=face.locations
        person_id=face.person_id
        for img in session.query(Images).all():
            if img.id==face.image_id and person_id==i :#and img.name=='427cd511-da04-4d18-8dcc-0359a327b4d7.jpg':

                image=cv2.imread('Data/'+img.name) 

                size=image.shape

                if (size[0]>1500 or size[1]>1500):
                    (y1,x2,y2,x1)=convert_array(location) 
                    x1=int(x1*4) 
                    y1=int(y1*4)
                    x2=int(x2*4)
                    y2=int(y2*4)
                elif (size[0]>1000 or size[1]>1000):
                    (y1,x2,y2,x1)=convert_array(location)
                    x1=int(x1*2) 
                    y1=int(y1*2)
                    x2=int(x2*2)
                    y2=int(y2*2)
                elif (size[0]>750 and size[1]>750):
                    (y1,x2,y2,x1)=convert_array(location)  
                    x1=int(x1*1.34) 
                    y1=int(y1*1.34)
                    x2=int(x2*1.34)
                    y2=int(y2*1.34)
                else:
                    (y1,x2,y2,x1)=convert_array(location)


                #(y1,x2,y2,x1)=convert_array(location)

                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(image, str(person_id), (x1+ 6, y1- 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 2)
                
                #if len(faces[person_id])>3:
                       
                 #   cv2.imwrite("Celeb_v1/Good/"+str(person_id)+"/"+img.name,image)
                       
                if len(faces[person_id])>2:
                         
                    #cv2.imwrite("Celeb_v1/Bad/"+str(person_id)+"/"+img.name,image)

                    cv2.imshow(str(i),image)
                    cv2.waitKey(1000)
                    cv2.destroyAllWindows()

                break

        #break